# 03. Coarse Grained Simulations

Coarse-graining a box of 500 methanol molecules from the all atom simulation given in `CG_tutorial/meth.gsd` file.

In [1]:
import tensorflow as tf
import hoomd
import hoomd.md
import hoomd.htf as htf
import numpy as np
import gsd, gsd.hoomd, pickle
import math
import re

In [2]:
### Graph building code ###

def make_train_graph(NN, rcut, system):

    #mamximum number of neighbors=0, graph will not compute forces to use in TF
    graph = htf.graph_builder(0, output_forces=False)
    assign_ops = []
    prints = []
    #get mapping from molecule index to particle index
    molecule_mapping = htf.find_molecules(system)

    for m in molecule_mapping:
        #ensure all molecules are of the same length
        assert len(m) == len(molecule_mapping[0])
    N = sum([len(m) for m in molecule_mapping])
    M = len(molecule_mapping)
    #number of atoms in a molecule=MN
    MN = len(molecule_mapping[0])
    # create one bead mapping. mapping matrices must be created for other types of mappings
    molecule_mapping_matrix = np.ones([1, MN], dtype=np.int)
    #create a mass-weighted M x N mapping operator 
    cg_mapping = htf.sparse_mapping([molecule_mapping_matrix for _ in molecule_mapping], molecule_mapping, system=system)
    assert cg_mapping.shape == (M, N)
    #calculate the center of mass of a CG bead
    mapped_pos = htf.center_of_mass(graph.positions[:,:3], cg_mapping, system, name='com-mapped-positions')
    #create the mapped neighbot list
    mapped_nlist = htf.compute_nlist(mapped_pos, rcut, NN, system, False)
    #create a non mass-weighted M x N mapping operator
    force_cg_mapping = htf.sparse_mapping([molecule_mapping_matrix for _ in molecule_mapping], molecule_mapping)
    #calculate the coarse grained forces
    mapped_force = tf.sparse.matmul(force_cg_mapping, graph.forces, name='mapped-forces')

    #save the graph
    graph.save('CG_tutorial/cg_model')                   

In [3]:
#====================RB_to_OPLS========================
#Taken from mbuild source code
def RB_to_OPLS(c0, c1, c2, c3, c4, c5):
    """Converts Ryckaert-Bellemans type dihedrals to OPLS type.
    Parameters
    ----------
    c0, c1, c2, c3, c4, c5 : Ryckaert-Belleman coefficients (in kcal/mol)
    Returns
    -------
    opls_coeffs : np.array, shape=(4,)
        Array containing the OPLS dihedrals coeffs f1, f2, f3, and f4
        (in kcal/mol)
    """

    f1 = (-1.5 * c3) - (2 * c1)
    f2 = c0 + c1 + c3
    f3 = -0.5 * c3
    f4 = -0.25 * c4
    return np.array([f1, f2, f3, f4])


In [4]:
### Hoomd-Sim code ###

with open('CG_tutorial/model.p', 'rb') as f:
    param_sys = pickle.load(f) #load parameters

g = gsd.hoomd.open('CG_tutorial/meth.gsd')
frame = g[0]
c = hoomd.context.initialize('--mode=cpu')
system = hoomd.init.read_gsd(filename='CG_tutorial/meth.gsd')
c.sorter.disable()
set_rcut=10.0

#call the graph building code
make_train_graph(128,10.0, system)

with htf.tfcompute('CG_tutorial/cg_model', device='CPU:0') as tfcompute:
    nlist = hoomd.md.nlist.cell()

    #set-up pppm
    charged = hoomd.group.all();
    pppm = hoomd.md.charge.pppm(nlist=nlist, group=charged)
    pppm.set_params(Nx=32, Ny=32, Nz=32, order=6, rcut=set_rcut)

    #set-up pair coefficients
    nlist.reset_exclusions(['1-2', '1-3', '1-4','body'])
    lj = hoomd.md.pair.force_shifted_lj(r_cut=set_rcut, nlist=nlist)
    forces = [lj]
    lj.pair_coeff.set("opls_156", "opls_156", sigma=2.5, epsilon=0.029999999999999995)
    lj.pair_coeff.set("opls_156", "opls_157", sigma=2.958039891549808, epsilon=0.044497190922573975)
    lj.pair_coeff.set("opls_156", "opls_154", sigma=2.79284800875379, epsilon=0.0714142842854285)
    lj.pair_coeff.set("opls_156", "opls_155", sigma=5.0, epsilon=0.0)
    lj.pair_coeff.set("opls_157", "opls_157", sigma=3.5, epsilon=0.066)
    lj.pair_coeff.set("opls_157", "opls_154", sigma=3.304542328371661, epsilon=0.10592450141492288)
    lj.pair_coeff.set("opls_157", "opls_155", sigma=5.916079783099616, epsilon=0.0)
    lj.pair_coeff.set("opls_154", "opls_154", sigma=3.12, epsilon=0.16999999999999998)
    lj.pair_coeff.set("opls_154", "opls_155", sigma=5.585696017507577, epsilon=0.0)
    lj.pair_coeff.set("opls_155", "opls_155", sigma=10.0, epsilon=0.0)

    #set-up bonds
    harmonic = hoomd.md.bond.harmonic()
    harmonic.bond_coeff.set("opls_156-opls_157", k=339.99999999999994, r0=1.09)
    harmonic.bond_coeff.set("opls_154-opls_157", k=319.99999999999994, r0=1.41)
    harmonic.bond_coeff.set("opls_154-opls_155", k=552.9999999999999, r0=0.945)

    #set-up angles
    harm_angle = hoomd.md.angle.harmonic()
    harm_angle.angle_coeff.set("opls_154-opls_157-opls_156", k=70.0, t0=1.91113553093)
    harm_angle.angle_coeff.set("opls_155-opls_154-opls_157", k=110.0, t0=1.89368223841)
    harm_angle.angle_coeff.set("opls_156-opls_157-opls_156", k=66.0, t0=1.88146493365)

    #set-up dihedrals
    dihedral = hoomd.md.dihedral.opls()
    f1,f2,f3,f4=RB_to_OPLS(0.22499999999999998, 0.6749999999999999, 0.0, -0.8999999999999999, 0.0, 0.0)
    dihedral.dihedral_coeff.set("opls_155-opls_154-opls_157-opls_156", k1=f1, k2=f2, k3=f3, k4=f4)

    group_all = hoomd.group.all()
    kT = 1.9872/1000

    #NVT Simulation in Hoomd
    im = hoomd.md.integrate.mode_standard(dt=5.0/489.0)
    nvt = hoomd.md.integrate.nvt(group=group_all, kT=298.15 * kT, tau=350 / 48.9)
    nvt.randomize_velocities(1234)

    #equilibrate
    hoomd.run(100)

    #communicate positions, neighbor list and forces to TensorFlow model
    tfcompute.attach(nlist, r_cut=set_rcut,period=1)

    #Hoomd production run
    hoomd.run(100)

print('Training Done')

HOOMD-blue v2.5.1 CUDA (10.0) SINGLE SSE SSE2 
Compiled: 03/07/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 6000 particles



Finding molecules...99.90%

Note: Backed-up CG_tutorial/cg_model previous model to CG_tutorial/cg_model/previous_model_0

notice(2): Started TF Session Manager.
-----
You are using PPPM. Please cite the following:
* D N LeBard, B G Levine, S A Barr, A Jusufi, S Sanders, M L Klein,